In [326]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import json
import time
import numpy as np

from selenium import webdriver ## Driver for Firefox, Chrome, Edge, etc.
from selenium.webdriver.common.by import By # Mode of locating html elements: ID, CSS_SELECTOR, XPATH, ...
from selenium.webdriver.support.select import Select

these first 3 are like batch collections right, so no delay necessary

In [327]:
def county_search():
    #go to start page and make into soup html obj
    url = "https://www.gelbeseiten.de/branchenbuch/staedte/schleswig-holstein/landkreise"
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    
    #find all counties
    lk = soup.find_all(class_="filterlist__item")
    landkreis_stack = [] #create empty stack to put counties into (not a stack tho)
    for e in lk:
        county_refined = e.text.lower()
        county_refined = county_refined.strip()
        landkreis_stack.append(county_refined)

    return landkreis_stack

In [328]:
def city_search(county_in_question):
    #pass relevant counties as argument and get its cities
    url = f"https://www.gelbeseiten.de/branchenbuch/staedte/schleswig-holstein/{county_in_question}"
    #print(url) #remove or comment later
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')

    places = soup.find_all(class_="boxteaser__title")

    city_stack = []
    for e in places:
        city_stack.append(e.text.lower()) #append all city names (for one county)

    return city_stack

In [329]:
def branch_collector(county,city):
    #now collect all types of buisness
    url = f"https://www.gelbeseiten.de/branchenbuch/staedte/schleswig-holstein/{county}/{city}"
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    soup_r = soup.find_all(class_="gc-teaser-tile__full-link")
    my_branch_list = [f"https://www.gelbeseiten.de{s.get("href")}" for s in soup_r]
    #some counties are so small they have no unternehmenslist, in this case just go via alle unternehmen
    if my_branch_list == []:
        alle_unternehmen_url = f"https://www.gelbeseiten.de/branchenbuch/staedte/schleswig-holstein/{county}/{city}/unternehmen"
        my_branch_list.append(alle_unternehmen_url)
    return my_branch_list

---

little detour: get more than 50 results

In [330]:
def expand_resultcount(url):
    #url = 'https://www.gelbeseiten.de/branchen/friseur/lübeck'
    driver = webdriver.Firefox()
    driver.get(url)

    driver.implicitly_wait(10)

    #manage cookie button
    try:
        accept_button = driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/div[2]/span[1]/a")
        accept_button.click()
    except:
        accept_button = None

    #find out how many results in total there are
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    result_totalCount = soup.find_all(id="loadMoreGesamtzahl")
    if len(result_totalCount) > 0:
        result_totalCount = int(result_totalCount[0].text) #get str w/ number, make into int
        result_totalCount = np.round((result_totalCount-50)/10) #because first 50 results are shown
    else:
        result_totalCount = 0 #f not more than 50 results available just skip while loop

    #click load more button as often as needed (each time 10 more results)
    load_more_clicked = 1
    while load_more_clicked <= result_totalCount:
        #driver.execute_script("arguments[0].scrollIntoView(true);", loadMore_button) #thanks chat gtp
        #loadMore_button.click()
        loadMore_button = driver.find_element(By.CSS_SELECTOR, "#mod-LoadMore--button")
        time.sleep(2)
        driver.execute_script("arguments[0].click();", loadMore_button) #use java script to execute button, since sometimes image ads or headers obscure view (see demo)
        load_more_clicked += 1

    #now use bs4 again since i already wrote this, just extract all hrefs
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    all_links = soup.find_all(id=re.compile(r'^treffer_\d+')) #each buisness is a 'treffer' (=hit) followed by _ and some digits
    my_buisnesses = [s.find("a")["href"] for s in all_links] #now get all hrefs for the buisnesses and return that list

    driver.quit()

    return my_buisnesses


---

In [331]:
#--- old version ---
#html = requests.get(b).text
#soup = BeautifulSoup(html, 'html.parser')
#ideally get all buisnesses of a branch, but right now somehow only 50 (and load next button doesnt work, i dont see any pagination)
#all_links = soup.find_all(id=re.compile(r'^treffer_\d+'))
#my_buisnesses = [s.find("a")["href"] for s in all_links]
#------

now we visit each website, so now we add a politeness delay

In [332]:
"""def collect_buisness_infos(branchen,buisnesslist):
    for b in branchen:
        #something like this url = "https://www.gelbeseiten.de/branchen/apotheke/kiel" is the url then
        
        #use function with selenium to expand search results from 50 to x, then use bs4 to collect all the hrefs for the buisnesses and return them as a list
        my_buisnesses = expand_resultcount(b)

        #not go through that list and get info on each buisness
        #use this broad try thing, because e.g. timeout errors occured at 126 elements and who knows which other issues might appear (also eduroam hates me)
        for elm in my_buisnesses:
            time.sleep(1)
            #for each element in list of all buisness per branch (in one city)
            try:
                html = requests.get(elm).text #go there
                soup = BeautifulSoup(html, 'html.parser')
                json_soup = soup.find_all(type="application/ld+json") #same as above
                if len(json_soup) > 0:
                    e = json_soup[len(json_soup)-1] #seems to be in last element always (i hope)
                    e = e.text
                    data = json.loads(e)
                else:
                    data = None
            except Exception as e:
                    error_type = type(e).__name__  # Get the name of the exception
                    error_message = str(e)         # Get the error message
                    data = {"website":elm, "error_type":error_type, "error_message":error_message}        

            buisnesslist.append(data)

    #return buisnesslist
    """

'def collect_buisness_infos(branchen,buisnesslist):\n    for b in branchen:\n        #something like this url = "https://www.gelbeseiten.de/branchen/apotheke/kiel" is the url then\n        \n        #use function with selenium to expand search results from 50 to x, then use bs4 to collect all the hrefs for the buisnesses and return them as a list\n        my_buisnesses = expand_resultcount(b)\n\n        #not go through that list and get info on each buisness\n        #use this broad try thing, because e.g. timeout errors occured at 126 elements and who knows which other issues might appear (also eduroam hates me)\n        for elm in my_buisnesses:\n            time.sleep(1)\n            #for each element in list of all buisness per branch (in one city)\n            try:\n                html = requests.get(elm).text #go there\n                soup = BeautifulSoup(html, \'html.parser\')\n                json_soup = soup.find_all(type="application/ld+json") #same as above\n              

In [320]:
def collect_buisness_infos2(branchen):
    buisnesslist = []
    for b in branchen:
        #something like this url = "https://www.gelbeseiten.de/branchen/apotheke/kiel" is the url then
        
        #use function with selenium to expand search results from 50 to x, then use bs4 to collect all the hrefs for the buisnesses and return them as a list
        my_buisnesses = expand_resultcount(b)

        #not go through that list and get info on each buisness
        #use this broad try thing, because e.g. timeout errors occured at 126 elements and who knows which other issues might appear (also eduroam hates me)
        for elm in my_buisnesses:
            time.sleep(1)
            #for each element in list of all buisness per branch (in one city)
            try:
                html = requests.get(elm).text #go there
                soup = BeautifulSoup(html, 'html.parser')
                json_soup = soup.find_all(type="application/ld+json") #same as above
                if len(json_soup) > 0:
                    e = json_soup[len(json_soup)-1] #seems to be in last element always (i hope)
                    e = e.text
                    data = json.loads(e)
                else:
                    data = None
            except Exception as e:
                    error_type = type(e).__name__  # Get the name of the exception
                    error_message = str(e)         # Get the error message
                    data = {"website":elm, "error_type":error_type, "error_message":error_message}        
            buisnesslist.append(data)

    return buisnesslist
    

first all types of buisness (branchen) get collected and only then the buisnesses 

In [321]:
"""sh_unternehmen = []

lk_stack = county_search()

for lk in lk_stack:
    city = city_search(lk)
    for c in city:
        my_branch_list = branch_collector(lk,c)
        collect_buisness_infos(my_branch_list,sh_unternehmen)

for kfs in ["kiel","lübeck","flensburg","neumünster"]:
    my_branch_list = branch_collector("kreisfrei",kfs)
    collect_buisness_infos(my_branch_list,sh_unternehmen) """

'sh_unternehmen = []\n\nlk_stack = county_search()\n\nfor lk in lk_stack:\n    city = city_search(lk)\n    for c in city:\n        my_branch_list = branch_collector(lk,c)\n        collect_buisness_infos(my_branch_list,sh_unternehmen)\n\nfor kfs in ["kiel","lübeck","flensburg","neumünster"]:\n    my_branch_list = branch_collector("kreisfrei",kfs)\n    collect_buisness_infos(my_branch_list,sh_unternehmen) '

In [322]:
#sh_unternehmen[0]

In [323]:
#len(sh_unternehmen)

---
---

try mongodb insertion


In [324]:
from pymongo import MongoClient

In [325]:
# Connect to MongoDB running in the Docker container
client = MongoClient('mongodb://localhost:27017/')

# Access a specific database (it will be created if it doesn't exist)
db = client['webscraper_yp']

# Access a specific collection (like a table in SQL)
# #db.create_collection('scraped_data')
collection = db['yp_data']


# --- my CODE ---

sh_unternehmen = []
lk_stack = county_search()

for lk in lk_stack:
    city = city_search(lk)
    for c in city:
        print(f"for city {c}, start inserting data:")
        my_branch_list = branch_collector(lk,c)
        sh_unternehmen = collect_buisness_infos2(my_branch_list) 
        collection.insert_many(sh_unternehmen) #insert data collected so far: for 1 city in 1 landkreis!
        print(f"some data inserted successfully for {c}!!!")

for kfs in ["kiel","lübeck","flensburg","neumünster"]:
    my_branch_list = branch_collector("kreisfrei",kfs)
    sh_unternehmen = collect_buisness_infos2(my_branch_list) 
    collection.insert_many(sh_unternehmen)

# ------

for city albersdorf holstein, start inserting data:


KeyboardInterrupt: 

In [333]:
j = branch_collector("dithmarschen","arkebek")

In [334]:
sh_unternehmen = collect_buisness_infos2(j) 

In [336]:
url = f"https://www.gelbeseiten.de/branchenbuch/staedte/schleswig-holstein/dithmarschen/arkebek"
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')
soup_r = soup.find_all(id="loadMoreGesamtzahl")

In [337]:
soup_r

[]

In [335]:
sh_unternehmen

[]

---
---

In [ ]:
# Store the JSON data in a file
with open("data_YP.txt", "w") as file:
    json.dump(sh_unternehmen, file, indent=4)

print("Data stored successfully!")

In [ ]:
# Store the JSON data in a file
with open("data_YP.json", "w") as file:
    json.dump(sh_unternehmen, file,indent=4)

print("Data stored successfully!")